# ICU survivor prediction
# Machine Learning, Exercise 1
### 王敏行 id:2018012386 wangmx18@mails.tsinghua.edu.cn
**All following codes originally generated by myself.**
Please see rm.md before running this file.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import accuracy_score

x_tr1 = np.array(pd.read_csv('data1forEx1to4/train1_icu_data.csv'))
y_tr1 = np.array(pd.read_csv('data1forEx1to4/train1_icu_label.csv'))
x_tr2 = np.array(pd.read_csv('data1forEx1to4/train2_icu_data.csv'))
y_tr2 = np.array(pd.read_csv('data1forEx1to4/train2_icu_label.csv'))
x_ts1 = np.array(pd.read_csv('data1forEx1to4/test1_icu_data.csv'))
y_ts1 = np.array(pd.read_csv('data1forEx1to4/test1_icu_label.csv'))
x_ts2 = np.array(pd.read_csv('data1forEx1to4/test2_icu_data.csv'))
y_ts2 = np.array(pd.read_csv('data1forEx1to4/test2_icu_label.csv'))

def get_index(x,y):
    id0 = np.array([index for (index, value) in enumerate(y) if value ==0])
    id1 = np.array([index for (index, value) in enumerate(y) if value ==1])
    s1=x[id0]
    s2=x[id1]
    return s1,s2


## Part1: Fisher's linear discriminant

In [2]:
def ScWi(A):
    m = np.mean(A, axis=0) 
    cov = np.zeros((A.shape[1],A.shape[1]))
    for sample in A:
        centered = sample-m
        centered = centered#.reshape(1,-1)
        # print(centered.shape)
        cov += centered*centered.reshape(-1,1)
        # print(cov.shape)
    return cov

def FLD(s1,s2):
    co1 = ScWi(s1)
    co2 = ScWi(s2)
    m1 = np.mean(s1, axis=0)
    m2 = np.mean(s2, axis=0)

    s_wi=co1+co2
    u,s,v = np.linalg.svd(s_wi)
    sw = np.dot(np.dot(v.T, np.linalg.inv(np.diag(s))), u.T)
    ws = np.dot(sw, m1-m2)
    # ws = np.linalg.inv(s_wi)
    w0=np.dot(ws.T,(m1+m2)/2)
    # print('ws',ws.shape)
    # print('w0',w0.shape)
    return ws,w0

# poor efficiency
# def judge(s,s1,s2):
#     ws,w0=FLD(s1,s2)
#     pos = np.dot(ws.T,s)
#     # print('pos', pos.shape,'s',s.shape)
#     return pos>w0

def judge(s,ws,w0):
    pos = np.dot(ws.T,s)
    # print('pos', pos.shape,'s',s.shape)
    return pos>w0


def pred(A,ws,w0):
    res = np.zeros((A.shape[0],2))
    for i in range(A.shape[0]):
        # print(i,judge(A[i,:],s1,s2).shape,judge(A[i,:],s1,s2))
        if judge(A[i,:],ws,w0):
            res[i,0] +=1
        else:
            res[i,1] +=1
    return res

s1,s2 = get_index(x_tr1,y_tr1)
ws,w0 = FLD(s1,s2)
res = pred(x_tr1,ws,w0)
print(res)

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [1. 0.]]


In [3]:
acc_tr1 = accuracy_score(y_tr1, res[:,1])
print('train1 acc:',acc_tr1)

train1 acc: 0.7944


In [4]:
s1,s2 = get_index(x_tr2,y_tr2)
ws,w0 = FLD(s1,s2)
res2 = pred(x_tr2,ws,w0)
acc2_tr2 = accuracy_score(y_tr2, res2[:,1])
print('train2 acc:',acc2_tr2)

train2 acc: 1.0


Test on test sets with model trained on train1 or train2 separately.
### Model trained on trainset 1:

In [5]:
s1,s2 = get_index(x_tr1,y_tr1)
ws,w0 = FLD(s1,s2)
pred1 = pred(x_ts1,ws,w0)
pred1 = pred1[:,1]
acc_ts1 = accuracy_score(y_ts1, pred1)
print('test1 acc:',acc_ts1,'\nerror rate:',1-acc_ts1)
np.savetxt('FLD_tr1_ts1.txt', pred1, fmt='%.0f')

test1 acc: 0.7757520510483136 
error rate: 0.22424794895168643


## Part2: perceptron

In [6]:
yp_tr1 = y_tr1*2-1 #0,1 to -1,1
yp_tr2 = y_tr2*2-1

def train_P(x,y, max_itr=300, lr0=0.005, decay_lr=0):
    w = np.random.randn(1,x.shape[1])
    b=0

    acc=[]
    x0 = []
    for i in range(max_itr):
        lr = lr0/(1+decay_lr*i)
        for j in range(x.shape[0]):
            if y[j]*(np.dot(w,x[j,:])+b)<=0:
                w = w + lr*y[j]*x[j,:]
                b += lr*y[j]
        if i%10:
            res = pred_P(x_tr2, w,b)
            acc.append(accuracy_score(y_tr2,res))
            x0.append(i)
        
    return w,b,x0,acc

def pred_P(x, w, b):
    pred = np.zeros((x.shape[0],1))
    for i in range(x.shape[0]):
        out = (np.dot(w,x[i,:])+b)
        if out>0:
            pred[i,0]=1
        elif out<0:
            pred[i,:]=0
    return pred